In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from evaluation import evaluate_matrices
from MatrixVectorizer import MatrixVectorizer

import sys
import os
# Add stp_gsr to the path
sys.path.append('stp_gsr')
from stp_gsr.src.models.stp_gsr import STPGSR
import hydra


In [24]:
class DictToObject:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            if isinstance(value, dict):
                setattr(self, key, DictToObject(value))
            else:
                setattr(self, key, value)

# Define the dictionary
config_dict = {
    "model": {
        "name": "stp_gsr",
        "target_edge_initializer": {
            "num_heads": 4,
            "edge_dim": 1,
            "hidden_dim": 256,
            "num_layers": 3,
            "dropout": 0.2,
            "beta": False
        },
        "dual_learner": {
            "in_dim": 1,
            "out_dim": 1,
            "num_heads": 1,
            "dropout": 0.2,
            "beta": False
        }
    },
    "dataset": {
        "name": "custom",
        "n_samples": 167,
        "n_source_nodes": 160,
        "n_target_nodes": 268,
        "node_feat_init": "adj",
        "node_feat_dim": 160  # Assuming this is derived from n_source_nodes
    },
    "experiment": {
        "defaults": [
            "_self_",
            {"dataset": "sbm"},
            {"model": "stp_gsr"}
        ],
        "n_epochs": 60,
        "batch_size": 16,
        "lr": 0.001,
        "log_val_loss": False,
        "base_dir": "results",
        "run_name": "run1",
        "kfold": {
            "n_splits": 3,
            "shuffle": True,
            "random_state": 42
        }
    }
}

torch.Size([112, 35778])


In [25]:
config = DictToObject(config_dict)

# Accessing attributes
print(config.model.name)  # Output: stp_gsr
print(config.model.target_edge_initializer.num_heads)  # Output: 4
print(config.dataset.n_samples)  # Output: 167
print(config.experiment.n_epochs)  # Output: 60
print(config.experiment.kfold.n_splits) 

stp_gsr
4
167
60
3


In [30]:
from stp_gsr.src.dataset import load_test
from stp_gsr.src.train import eval

ImportError: cannot import name 'load_test' from 'stp_gsr.src.dataset' (/vol/bitbucket/zz1224/dgl/dgl-project/stp_gsr/src/dataset.py)

In [ ]:
# Load the test data and the model, and evaluate the model on the test data.
sys.path.append('stp_gsr')
LR_TEST_DATA_FILE_NAME = 'data/lr_test.csv'
MODEL_FILE_NAME = 'stp_gsr/results/stp_gsr_modified/final/model.pth'

# Load the test data
# test_data = pd.read_csv(LR_TEST_DATA_FILE_NAME)

# Load the model
model = STPGSR(config) 
model.load_state_dict(torch.load(MODEL_FILE_NAME))
model.eval()

# Change test data to torch tensor
# test_data = torch.tensor(test_data.values, dtype=torch.float32)

test_data = load_test(config)

# Predict the test data
y_pred = model()
print(y_pred.shape)



TargetEdgeInitializer(
  (convs): ModuleList(
    (0): TransformerConv(160, 64, heads=4)
    (1): TransformerConv(256, 64, heads=4)
    (2): TransformerConv(256, 67, heads=4)
  )
  (bns): ModuleList(
    (0-1): 2 x GraphNorm(256)
    (2): GraphNorm(268)
  )
  (residual_proj): Sequential(
    (0): Linear(in_features=160, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=268, bias=True)
  )
)
--------------------------------------------------
DualGraphLearner(
  (conv1): TransformerConv(1, 1, heads=1)
  (bn1): GraphNorm(1)
)


TypeError: STPGSR.forward() missing 1 required positional argument: 'target_mat'

In [12]:
# flatten
y_pred = y_pred.detach().numpy()
y_pred = y_pred.flatten()
print(y_pred.shape)

(4007136,)


In [13]:
# Convert tensor to csv
df = pd.DataFrame({
    'ID': np.arange(1, len(y_pred)+1),
    'Predicted': y_pred
})

df.to_csv('prediction.csv', index=False)
print('Prediction saved to prediction.csv')

Prediction saved to prediction.csv
